In [1]:
import numpy as np
import pandas as pd
import seaborn as sns
import re
from tabulate import tabulate

In [ ]:
from tkinter import *

#main window
class MainWindow:
    #get result function
    def getresult(self):
        import math
        import requests
        #read csv
        filename = 'Sentiment_Reviews_Agg.csv'
        dataor = pd.read_csv(filename,encoding='ISO-8859-1')
        #convert data type of sentiment score to %
        lists=dataor["sentiment_compound_polarity"].tolist()
        for i in range(len(lists)):
            lists[i]=str(int(lists[i]*100))+"%"
        dataor["sentiment"]=pd.DataFrame(lists)
        
        listind=dataor.index.tolist()
        
        #create a new dataframe to save results
        df1 = pd.DataFrame(columns=dataor.columns)
        df1 = df1.fillna(0) # with 0s rather than NaNs
        
        #use Bing Map API to get latitude and longitude
        location=self.entry_loc.get()
        apikey="An1LS453eKDhd2U06Fx-uUcizTOBGO00Ib2pZ6M1jyr1ux9KI6ukDdkTAiIoZP0r"
        url="http://dev.virtualearth.net/REST/v1/Locations?q=%s&output=json&key=%s" % (location,apikey)
        response = requests.get(url).json()
        lat,lng = response['resourceSets'][0]['resources'][0]['geocodePoints'][0]['coordinates'][0],response['resourceSets'][0]['resources'][0]['geocodePoints'][0]['coordinates'][1]
        
        #calculate distance
        radius = 6371 # km
        radius_filter=float(self.entry_rad.get())
        rprice=float(self.entry_pri.get())
        
        for elem in listind:
            lat2 = float(dataor.iloc[elem,5])
            lng2= float(dataor.iloc[elem,6])

            dlat = math.radians(lat2-lat)
            dlon = math.radians(lng2-lng)
            a = math.sin(dlat/2) * math.sin(dlat/2) + math.cos(math.radians(lat)) \
                * math.cos(math.radians(lat2)) * math.sin(dlon/2) * math.sin(dlon/2)
            c = 2 * math.atan2(math.sqrt(a), math.sqrt(1-a))
            d = radius * c
    
            dataor["distance"]=d
        
            #select suitable listings and save them to df1
            if d < radius_filter and rprice-20 < dataor.iloc[elem,13] < rprice+20:
                df1=df1.append(dataor.iloc[elem])

        
        #sorted by sentiment scores
        df1.sort_values("sentiment_compound_polarity",inplace=True,ascending=False)   
        
        #drop useless columns
        df1=df1.drop(["Description","ZipCode","Latitude","Longitude","bathrooms","bedrooms","reviews_per_month","review_scores_rating","sentiment_compound_polarity"],axis=1)
        
        #visualization
        table = tabulate(df1,headers='keys')
        self.text.insert('end', table)  

    
    # GUI window commands
    def __init__(self):
        self.frame = Tk()
        
        #window title
        self.frame.title('Airbnb Search')
        
        #window geometry
        self.frame.geometry('1400x800')
        
        #add labels 
        self.label_loc = Label(self.frame,text = "Location: ")
        self.label_rad = Label(self.frame,text = "Radius: ")
        self.label_pri = Label(self.frame,text = "Price: ")
        
        #add entry part
        self.entry_loc = Entry(self.frame)
        self.entry_rad = Entry(self.frame)
        self.entry_pri = Entry(self.frame)
        
        #add text part
        self.text = Text(self.frame,width=200)
        
        #add button 
        self.button_search = Button(self.frame,text = "Search",width = 10,command= self.getresult )
        self.button_cancel = Button(self.frame,text = "Cancel",width = 10 )
        
        #grid them
        self.label_loc.grid(row = 0,column = 6)
        self.label_rad.grid(row = 1,column = 6)
        self.label_pri.grid(row = 2,column = 6)
        
        self.entry_loc.grid(row = 0,column = 7)
        self.entry_rad.grid(row = 1,column = 7)
        self.entry_pri.grid(row = 2,column = 7)
        
        self.button_search.grid(row = 3,column = 6)
        self.button_cancel.grid(row = 3,column = 7)
        
        self.text.grid(row=4,columnspan=20)
        
        self.frame= mainloop()

frame = MainWindow()

